# Connected Devices Extraction

This notebook determines from the connectivity events which devices are connected in a given time frame

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import aux_balena as aux
from tqdm import tqdm_notebook
%matplotlib inline
from datetime import datetime, timedelta
import pickle

In [2]:
dflog = aux.read_log('connectivity_events_clean_Final.csv')

Mem. usage decreased to 143.23 Mb (51.5% reduction)
Log read in 19.4 s
Converting to timestamp field:  timestamp
Done!


In [3]:
dflog.head()

timestamp  device_id  user_id  server_id  connected
0 2015-05-31 14:30:00          1        1          2       True
1 2015-06-02 10:30:00          1        1          2      False
2 2015-05-31 14:30:00          2        2          2       True
3 2015-06-05 20:30:00          2        2          2      False
4 2015-05-31 14:30:00          3        1          2       True

## Functions to extract connected devices

Purpose is to report all connected devices between date1 and date2.
The algorithm detects a connected devices if:
- the last point before date1 in the time series is connected

OR

- there is at least one connected point between date1 and date2

In [4]:
# auxiliary function
def get_last_point(df):
    if len(df)==0:
        return None
    else:
        return df.iloc[-1]

In [5]:
def get_active_devices(df, day1, day2):
    # returns device id's for all active devices between day1 and day2
    # day1 and day2 may be in any order
    
    # order start and end time
    start = min(day1, day2)
    end = max(day1, day2)
    
    # get activation of last data point before start time
    dcut = df[df.timestamp < start]
    #
    all_prev=dcut.groupby('device_id')['connected'].apply(get_last_point)
    #
    # get activation between start-end
    # time filter
    dinner = df[(df.timestamp>=start) & (df.timestamp<=end)]
    all_inner = dinner.groupby('device_id')['connected'].any()
    #
    # align series
    all_prev_alg, all_inner_alg = all_prev.align(all_inner, join = 'outer', fill_value=False)
    # perform OR operation
    all_conns = (all_prev_alg | all_inner_alg)
    # get device id's
    connected_devices = all_conns[all_conns==True].index.values
    #
    return connected_devices
    

In [6]:
def get_1day_active_devices(df, ref_day):
    # returns active devices for the day of choice
    day2 = ref_day + timedelta(days=1)
    #
    return get_active_devices(df, ref_day, day2)

In [8]:
def get_28pastdays_active_devices(df, ref_day):
    # returns active devices in the 28 days PRECEDING the reference date
    day2 = ref_day - timedelta(days=28)
    #
    return get_active_devices(df, ref_day, day2)

*Date limits*

In [11]:
start = datetime(2015,5,31) # values taken from the range of dates in log database
end = datetime(2018,3,1)

ts = np.arange(start, end, timedelta(days=1)).astype(datetime)
print('Number of days:', len(ts))
print('Last day in the series:', ts[-1])

Number of days: 1005
Last day in the series: 2018-02-28 00:00:00


### Extract 1day active devices

In [12]:
# dictionary of connected devices day by day
conn_dict_1day={}

for day in tqdm_notebook(ts):
    # get devs id's
    connected_dev_1day = get_1day_active_devices(dflog, day)
    # convert timedate to string for dictionary key
    day_string =day.strftime('%Y-%m-%d')
    # append to dict
    conn_dict_1day[day_string]= connected_dev_1day

Dump results to binary file with pickle

In [13]:
pickle.dump(conn_dict_1day, open('connected_devices_1day.pk','wb'))

### Extract 28 past days active devices

In [14]:
conn_dict_28day={}
for day in tqdm_notebook(ts):
    # get devs id's
    connected_dev_28day = get_28pastdays_active_devices(dflog, day)
    # convert timedate to string for dictionary key
    day_string =day.strftime('%Y-%m-%d')
    # append to dict
    conn_dict_28day[day_string]= connected_dev_28day

In [15]:
pickle.dump(conn_dict_28day, open('connected_devices_28days.pk','wb'))